# Import

In [18]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn import model_selection
from sklearn.metrics import make_scorer

import warnings
warnings.filterwarnings('ignore')
np.random.seed(13)

# Load dataset

In [9]:
X_train = pd.read_csv("/scratch/nathanya.queby/mlpc/train_dataset.csv")
X_train = X_train.drop(['pianist_id', 'segment_id', 'snippet_id'], axis=1)
X_train

,essentia_dissonance_mean,essentia_dissonance_stdev,essentia_dynamic_complexity,essentia_loudness,essentia_onset_rate,essentia_pitch_salience_mean,essentia_pitch_salience_stdev,essentia_spectral_centroid_mean,essentia_spectral_centroid_stdev,essentia_spectral_complexity_mean,...,mirtoolbox_hcdf_pct_10,mirtoolbox_hcdf_pct_90,mirtoolbox_irregularity,mirtoolbox_keyclarity,mirtoolbox_mode,mirtoolbox_novelty_mean,mirtoolbox_novelty_pct_10,mirtoolbox_pulseclarity,mirtoolbox_roughness_mean,mirtoolbox_roughness_std
0,0.192237,0.059404,2.040252,16079768.0,0.800000,0.677256,0.095846,1290.972900,936.912476,30.490740,...,0.077880,0.357389,0.685941,0.809092,-0.092027,0.041443,0.00000,0.169743,0.011449,0.004161
1,0.143425,0.064204,3.138845,35489248.0,2.600000,0.514913,0.092372,1133.526733,286.292847,28.111111,...,0.113479,0.488570,0.911455,0.767972,-0.153564,0.113378,0.00000,0.355861,0.006205,0.003194
2,0.158810,0.059129,3.567908,40922732.0,1.400000,0.596779,0.116061,1076.780151,251.776215,29.354166,...,0.086685,0.481198,0.367500,0.523341,-0.083029,0.053186,0.00002,0.232454,0.007335,0.004063
3,0.168547,0.049648,2.329854,51921612.0,1.600000,0.556160,0.143420,1008.112183,196.336700,29.375000,...,0.115372,0.488982,1.002424,0.376181,-0.172612,0.063815,0.00000,0.140930,0.007345,0.003917
4,0.196514,0.052237,2.566071,66282424.0,2.200000,0.584508,0.100911,1063.062866,184.782257,28.534721,...,0.129901,0.513714,0.582648,0.699472,0.039179,0.088127,0.00000,0.091799,0.009359,0.003582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2048,0.202177,0.050760,2.840705,51977460.0,1.800000,0.688834,0.090235,866.423889,133.063477,29.842592,...,0.214162,0.682742,0.250832,0.806177,-0.042110,0.145387,0.00000,0.269012,0.014044,0.004991
2049,0.174454,0.063426,5.202462,9471357.0,3.200000,0.661253,0.107627,828.768005,234.908127,30.296297,...,0.092215,0.533371,0.969141,0.861597,-0.290169,0.091809,0.00000,0.353269,0.011126,0.004529
2050,0.161152,0.074724,4.605277,7378860.5,2.600000,0.645111,0.093705,841.285645,226.538666,29.761574,...,0.052282,0.702560,0.256100,0.829834,-0.254077,0.065565,0.00000,0.319188,0.009205,0.004808
2051,0.192846,0.056795,2.563405,22641664.0,3.200000,0.680604,0.094989,716.426025,181.553772,29.881945,...,0.178608,0.607663,0.255105,0.483378,-0.079091,0.188159,0.00000,0.133573,0.013538,0.004881


In [10]:
y_train = pd.read_csv("/scratch/nathanya.queby/mlpc/train_targets.csv", header=None)
y_train = y_train[0].values.astype("int64")
print(y_train.shape)
y_train

(2053,)


array([3, 3, 3, ..., 1, 1, 1])

In [11]:
X_test = pd.read_csv("/scratch/nathanya.queby/mlpc/test_dataset.csv")
X_test = X_test.drop(['pianist_id', 'segment_id', 'snippet_id'], axis=1)
X_test

,essentia_dissonance_mean,essentia_dissonance_stdev,essentia_dynamic_complexity,essentia_loudness,essentia_onset_rate,essentia_pitch_salience_mean,essentia_pitch_salience_stdev,essentia_spectral_centroid_mean,essentia_spectral_centroid_stdev,essentia_spectral_complexity_mean,...,mirtoolbox_hcdf_pct_10,mirtoolbox_hcdf_pct_90,mirtoolbox_irregularity,mirtoolbox_keyclarity,mirtoolbox_mode,mirtoolbox_novelty_mean,mirtoolbox_novelty_pct_10,mirtoolbox_pulseclarity,mirtoolbox_roughness_mean,mirtoolbox_roughness_std
0,0.146967,0.056205,2.019706,42130144.0,2.600000,0.560116,0.115359,1050.947632,181.327225,28.354166,...,0.126654,0.493207,0.740554,0.655391,-0.002827,0.143144,0.000000,0.209162,0.007467,0.003986
1,0.195762,0.054038,1.650751,65885464.0,4.373398,0.576964,0.095918,1104.464722,413.591125,27.740311,...,0.154411,0.455670,0.447020,0.661684,0.074573,0.141053,0.000000,0.149663,0.011406,0.003618
2,0.225943,0.058967,1.339994,156672624.0,3.000000,0.648186,0.087664,1066.024902,157.144470,29.175926,...,0.139321,0.470631,0.274534,0.675268,-0.280677,0.081721,0.000000,0.133185,0.015127,0.004930
3,0.169778,0.065160,4.615561,67287496.0,1.852280,0.607226,0.102608,1074.738159,471.420258,28.938503,...,0.130897,0.472980,0.736538,0.813011,0.207040,0.069847,0.000000,0.110838,0.011128,0.006774
4,0.226559,0.065521,1.705109,100496784.0,3.400000,0.621009,0.092719,1254.885254,199.422180,28.270834,...,0.139423,0.442981,0.426168,0.781752,0.157758,0.094246,0.000000,0.157236,0.014241,0.004307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,0.142812,0.065007,2.547269,16216005.0,3.000000,0.555797,0.126262,783.319885,152.581253,27.655092,...,0.090617,0.473097,0.836046,0.926008,0.099228,0.035435,0.000009,0.116657,0.008016,0.004037
225,0.161294,0.059252,2.995837,29173024.0,3.400000,0.600145,0.110782,696.067627,170.105576,28.902779,...,0.199239,0.552169,0.379475,0.636458,-0.029950,0.153426,0.000000,0.185843,0.009054,0.004111
226,0.265052,0.099702,1.549239,35251556.0,3.400000,0.610611,0.109219,1216.717407,287.146179,28.847221,...,0.158637,0.476680,0.403751,0.652105,0.127109,0.110063,0.004220,0.127996,0.018429,0.007174
227,0.170429,0.064642,3.810143,13938461.0,2.000000,0.669719,0.075449,893.238342,151.110596,27.055555,...,0.045320,0.639122,0.263926,0.744871,0.236403,0.097617,0.000000,0.156304,0.008120,0.005910


In [12]:
y_test = pd.read_csv("/scratch/nathanya.queby/mlpc/test_targets.csv", header=None)
y_test = y_test[0].values.astype("int64")
print(y_test.shape)
y_test

(229,)


array([3, 1, 3, 4, 4, 3, 3, 2, 2, 3, 2, 4, 3, 3, 3, 1, 1, 4, 3, 3, 2, 2,
       2, 2, 4, 1, 1, 1, 1, 1, 1, 4, 2, 2, 2, 2, 3, 1, 3, 4, 4, 4, 3, 1,
       4, 3, 3, 2, 4, 4, 3, 3, 1, 3, 1, 1, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1,
       2, 3, 4, 2, 1, 1, 4, 4, 3, 2, 2, 2, 3, 3, 3, 1, 4, 4, 4, 4, 4, 2,
       1, 1, 1, 1, 4, 4, 3, 3, 3, 2, 2, 3, 4, 3, 4, 4, 3, 4, 4, 1, 4, 4,
       4, 4, 1, 1, 2, 2, 2, 3, 2, 2, 2, 3, 3, 4, 4, 4, 4, 1, 1, 2, 1, 3,
       3, 2, 4, 4, 1, 1, 4, 4, 1, 4, 3, 3, 2, 2, 2, 3, 2, 2, 3, 3, 3, 3,
       4, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 4, 2, 3, 2, 3, 3, 3,
       3, 3, 4, 4, 1, 1, 4, 3, 3, 3, 4, 4, 4, 4, 4, 3, 3, 3, 2, 2, 2, 3,
       4, 1, 1, 1, 4, 4, 3, 3, 3, 1, 4, 1, 1, 4, 4, 4, 4, 4, 2, 2, 3, 1,
       1, 1, 1, 1, 4, 4, 4, 4, 4])

# Create train & test set

In [13]:
new_X = pd.concat([X_train, X_test])
new_X

,essentia_dissonance_mean,essentia_dissonance_stdev,essentia_dynamic_complexity,essentia_loudness,essentia_onset_rate,essentia_pitch_salience_mean,essentia_pitch_salience_stdev,essentia_spectral_centroid_mean,essentia_spectral_centroid_stdev,essentia_spectral_complexity_mean,...,mirtoolbox_hcdf_pct_10,mirtoolbox_hcdf_pct_90,mirtoolbox_irregularity,mirtoolbox_keyclarity,mirtoolbox_mode,mirtoolbox_novelty_mean,mirtoolbox_novelty_pct_10,mirtoolbox_pulseclarity,mirtoolbox_roughness_mean,mirtoolbox_roughness_std
0,0.192237,0.059404,2.040252,16079768.0,0.8,0.677256,0.095846,1290.972900,936.912476,30.490740,...,0.077880,0.357389,0.685941,0.809092,-0.092027,0.041443,0.000000,0.169743,0.011449,0.004161
1,0.143425,0.064204,3.138845,35489248.0,2.6,0.514913,0.092372,1133.526733,286.292847,28.111111,...,0.113479,0.488570,0.911455,0.767972,-0.153564,0.113378,0.000000,0.355861,0.006205,0.003194
2,0.158810,0.059129,3.567908,40922732.0,1.4,0.596779,0.116061,1076.780151,251.776215,29.354166,...,0.086685,0.481198,0.367500,0.523341,-0.083029,0.053186,0.000020,0.232454,0.007335,0.004063
3,0.168547,0.049648,2.329854,51921612.0,1.6,0.556160,0.143420,1008.112183,196.336700,29.375000,...,0.115372,0.488982,1.002424,0.376181,-0.172612,0.063815,0.000000,0.140930,0.007345,0.003917
4,0.196514,0.052237,2.566071,66282424.0,2.2,0.584508,0.100911,1063.062866,184.782257,28.534721,...,0.129901,0.513714,0.582648,0.699472,0.039179,0.088127,0.000000,0.091799,0.009359,0.003582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,0.142812,0.065007,2.547269,16216005.0,3.0,0.555797,0.126262,783.319885,152.581253,27.655092,...,0.090617,0.473097,0.836046,0.926008,0.099228,0.035435,0.000009,0.116657,0.008016,0.004037
225,0.161294,0.059252,2.995837,29173024.0,3.4,0.600145,0.110782,696.067627,170.105576,28.902779,...,0.199239,0.552169,0.379475,0.636458,-0.029950,0.153426,0.000000,0.185843,0.009054,0.004111
226,0.265052,0.099702,1.549239,35251556.0,3.4,0.610611,0.109219,1216.717407,287.146179,28.847221,...,0.158637,0.476680,0.403751,0.652105,0.127109,0.110063,0.004220,0.127996,0.018429,0.007174
227,0.170429,0.064642,3.810143,13938461.0,2.0,0.669719,0.075449,893.238342,151.110596,27.055555,...,0.045320,0.639122,0.263926,0.744871,0.236403,0.097617,0.000000,0.156304,0.008120,0.005910


In [14]:
new_y = np.append(y_train, y_test)
new_y.shape

(2282,)

In [15]:
new_test = pd.read_csv("/scratch/nathanya.queby/mlpc/task_4_test_dd4bd32b08b776e6_daf99ad_generic.csv")
new_test = new_test.drop(['pianist_id', 'segment_id', 'snippet_id'], axis=1)
features_to_exclude = ['midlevel_features_tonal_stability','midlevel_features_dissonance','essentia_strong_peak_stdev','librosa_chroma_pct_10_3','librosa_chroma_pct_50_0','librosa_chroma_pct_50_1','librosa_chroma_pct_50_2','librosa_chroma_pct_50_3','librosa_chroma_pct_50_4','librosa_chroma_pct_50_5','librosa_chroma_pct_50_6','librosa_chroma_pct_50_7','librosa_chroma_pct_50_8','librosa_chroma_pct_50_9','librosa_chroma_pct_50_10','librosa_chroma_pct_50_11','mirtoolbox_hcdf_pct_50','mirtoolbox_novelty_pct_50','mirtoolbox_novelty_std','mirtoolbox_novelty_pct_90','mirtoolbox_roughness_pct_10','mirtoolbox_roughness_pct_50','mirtoolbox_roughness_pct_90','mirtoolbox_dynamics_mean','mirtoolbox_dynamics_pct_10','mirtoolbox_dynamics_pct_50','mirtoolbox_dynamics_pct_90','librosa_mfcc_std_1','librosa_mfcc_std_7','librosa_mfcc_std_8','librosa_mfcc_std_9','librosa_mfcc_std_10','librosa_mfcc_std_11','librosa_mfcc_pct_50_8','librosa_mfcc_pct_50_9','librosa_mfcc_pct_50_10','librosa_mfcc_pct_50_11','librosa_mfcc_pct_50_6','librosa_mfcc_pct_50_7','librosa_mfcc_pct_50_1','librosa_mfcc_pct_90_3','librosa_mfcc_pct_90_4','librosa_mfcc_pct_90_5','librosa_mfcc_pct_10_8','librosa_mfcc_pct_10_6','librosa_mfcc_pct_50_5','librosa_mfcc_pct_10_5','librosa_mfcc_pct_50_4','librosa_mfcc_pct_10_4','librosa_mfcc_pct_10_3','librosa_mfcc_pct_50_3','librosa_mfcc_pct_10_1','librosa_mfcc_pct_10_2','librosa_mfcc_pct_50_2','librosa_mfcc_pct_90_2','librosa_mfcc_pct_10_0','librosa_mfcc_pct_50_0','librosa_mfcc_pct_90_0','librosa_mfcc_mean_0','librosa_mfcc_mean_2']

new_test = new_test.loc[:, ~new_test.columns.isin(new_test.filter(features_to_exclude))]
new_test

,essentia_dissonance_mean,essentia_dissonance_stdev,essentia_dynamic_complexity,essentia_loudness,essentia_onset_rate,essentia_pitch_salience_mean,essentia_pitch_salience_stdev,essentia_spectral_centroid_mean,essentia_spectral_centroid_stdev,essentia_spectral_complexity_mean,...,mirtoolbox_hcdf_pct_10,mirtoolbox_hcdf_pct_90,mirtoolbox_irregularity,mirtoolbox_keyclarity,mirtoolbox_mode,mirtoolbox_novelty_mean,mirtoolbox_novelty_pct_10,mirtoolbox_pulseclarity,mirtoolbox_roughness_mean,mirtoolbox_roughness_std
0,0.229886,0.062064,4.818434,93184008.0,3.600000,0.614987,0.090779,1365.162964,331.913849,28.465279,...,0.069374,0.278847,1.029432,0.877971,-0.332906,0.104101,0.000000,0.046520,0.010760,0.005877
1,0.178258,0.062866,3.003938,109259888.0,4.000000,0.564438,0.094290,1215.545288,271.362946,27.719908,...,0.063834,0.384670,1.150170,0.763541,0.105649,0.089613,0.000000,0.175834,0.006592,0.002608
2,0.209623,0.063750,1.983438,98162960.0,4.600000,0.553583,0.111736,1339.914185,255.108398,28.067129,...,0.148812,0.518528,0.301306,0.659473,-0.148198,0.192730,0.000000,0.153210,0.007164,0.003635
3,0.215004,0.061839,1.470698,72971816.0,3.915029,0.642106,0.092053,1143.090576,483.887421,28.758064,...,0.181710,0.545455,0.220104,0.760754,0.172775,0.068173,0.007496,0.147771,0.012342,0.004185
4,0.187017,0.072593,2.613981,80688832.0,2.600000,0.564319,0.104814,1148.319458,238.749054,28.587963,...,0.111090,0.420087,1.065051,0.567374,-0.105469,0.067202,0.000000,0.283948,0.008589,0.003603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,0.239235,0.048011,1.178853,50233408.0,2.516549,0.640804,0.073967,1298.177979,268.525360,27.625000,...,0.079891,0.570405,0.583360,0.788715,-0.115055,0.174047,0.000000,0.157421,0.012897,0.003446
2081,0.293661,0.060317,2.598517,178390704.0,2.400000,0.676194,0.080788,1602.075806,396.212982,29.527779,...,0.114611,0.403662,0.339105,0.612967,0.038338,0.074019,0.000000,0.314337,0.015907,0.005273
2082,0.272440,0.058219,1.851364,178757696.0,4.600000,0.681900,0.081200,1246.282837,252.229889,30.159721,...,0.158639,0.442483,0.369082,0.502579,-0.087977,0.136694,0.000000,0.268032,0.016610,0.004190
2083,0.273417,0.060010,1.930694,285332608.0,2.000000,0.708230,0.066477,1536.803711,552.418701,29.668982,...,0.082022,0.262809,0.758210,0.803343,0.218998,0.109739,0.000000,0.241217,0.015410,0.004318


# Create custom scoring function

In [16]:
def calc_revenue(y_true, y_pred):
    confusions = confusion_matrix(y_pred, y_true)
    # calculate revenue
    gain_matrix = np.array([
        [5, -5, -5, 2],
        [-5, 10, 2, -5],
        [-5, 2, 10, -5],
        [2, -5, -2, 5]])
    revenue = (confusions * gain_matrix).sum()
    return revenue

In [17]:
revenue = make_scorer(calc_revenue, greater_is_better=True)

# XGB parameter tuning

In [ ]:
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.001, 0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBClassifier(seed = 13)
xgbb = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring=revenue, 
                   cv=10,
                   verbose=2,
                   n_jobs=20)

xgbb.fit(new_X, new_y)

In [ ]:
print("Best parameters:", xgbb.best_params_)
print("Highest revenue: ", xgbb.best_score_)

# Use best parameters

In [ ]:
xgbbb = xgb.XGBClassifier(colsample_bytree=0.3, learning_rate=0.001, max_depth=10, n_estimators=100, seed=13)
cv_results = model_selection.cross_validate(xgbbb, new_X, new_y,
                    scoring=revenue)
xgbbb = xgbbb.fit(new_X, new_y)

In [ ]:
y_pred = xgbbb.predict(new_test)
print(y_pred.shape)
y_pred

In [ ]:
result_df = pd.read_csv("/scratch/nathanya.queby/mlpc/task_4_test_dd4bd32b08b776e6_daf99ad_generic.csv")[['pianist_id', 'segment_id', 'snippet_id']]
result_df["quadrant"] = y_pred
result_df

# Save result to csv

In [ ]:
result_df.to_csv("result_xgb.csv", index=False)